# Vector elements & Betti numbers

We have already seen the Lagrange and the DG finite element spaces. They are both spaces of *scalar*-valued functions. How can we approximate solutions of boundary value problems that are *vector fields*?

We could certainly use a Cartesian product of  scalar finite element spaces to approximate a vector field component by component. However, distinctly better methods can often be constructed using other vector finite element spaces, such as the *Raviart-Thomas* space or the *Nedelec* space, both spaces of vector fields we shall see in this notebook.

We will expand on their utility in certain  boundary value problems in later lectures. For now, the goal is to introduce you to computing with them and to show you that there might be more than just utilitarian reasons for studying these finite elements. They are connected through rich and beautiful discrete structures. As a prelude to such things, we will see in this notebook how a topological invariant emerges when considering curl as an operator from the Nedelec space in two space dimensions.

## The Raviart-Thomas space

$\newcommand{\om}{\varOmega}
\newcommand{\oh}{\varOmega_h}
\newcommand{\dive}{\mathop{\mathrm{div}}}
\newcommand{\R}{\mathbb{R}}
$
The Raviart-Thomas (RT) finite element space in two space dimensions is defined by 
$$
R_{h, p} =
\left\{
q\in H(\dive, \om) : \; \text{ for all elements } K \in \oh, \quad  q|_K \in P_p(K)^2 + \begin{pmatrix} x \\ y \end{pmatrix} P_p(K) \; \right\}
$$
for any $p \ge 0$. Here $P_p(K)$ denote the set of polynomials of degree at most $p$.
The distributional divergence of a piecewise smooth vector field was calculated in an earlier lecture, from which it follows that vector fields $q$ in  $R_{h, p}$ satisfy 
&LeftDoubleBracket; $q\cdot n$ &RightDoubleBracket; $=0$, i.e., the jump of the normal component of $q$ across element interfaces vanishes. Therefore, plots of any function in the RT space should show a *continuously varying normal component* across the interior mesh facets.

The continuity of the normal component is a useful property to have when we know that the  function being approximated has that same normal continuity property. Often, when $q: \om \to \R^2$ is a vector field representing a physical *flux*, its normal component remains smooth across a material interface even when its other components jump across the interface.

#### Interpolation

As an example, consider approximating the following vector field  using the RT space:  
$$
q = 
\left\{
\begin{aligned}
& (\sin(x), y),  && x \le 1/2,
\\
& (\sin(x), 1-y), && x > 1/2.
\end{aligned}
\right.
$$
Across the interface $x=1/2$, its normal component (the $x$-component in this case) is continuous, while its tangential component is discontinuous. Let's plot $q$ after making subdomains split across the $x=1/2$ line. 

In [ ]:
import ngsolve as ng
import numpy as np
from ngsolve.webgui import Draw
from netgen.occ import OCCGeometry ,WorkPlane, Rectangle, Glue
from ngsolve import x, y, sin, cos, GridFunction, div

# Make left and right subdomains
wp = WorkPlane()
lft = wp.Rectangle(1/2, 1).Face()
rgt = wp.MoveTo(1/2,0).Rectangle(1/2, 1).Face()
lft.faces.name = 'lft'
rgt.faces.name = 'rgt'
geo = Glue([lft,rgt])
Draw(geo)
mesh = ng.Mesh(OCCGeometry(geo, dim=2).GenerateMesh(maxh=1/4))

In [ ]:
# Define q piecewise on the left and right subdomains
q = mesh.MaterialCF({'lft': (sin(x), y),
                     'rgt': (sin(x), 1-y)})

The normal component (which is the $x$ component in this case) of $q$ is smooth:

In [ ]:
Draw(q[0], mesh);

The tangential component of $q$ is discontinuous across the middle interface.

In [ ]:
Draw(q[1], mesh);

We can also plot $q$ as a vector field.

In [ ]:
Draw(q, mesh, vectors={'grid_size': 15});

If we try to interpolate such a function using a product of Lagrange spaces, then the discontinuity at the interface will generate large interpolation errors. Instead, knowing that the normal component is continuous, we can try to interpolate the function into the RT space.  This will liberate the tangential component of the approximation from continuity constraints, while preserving the normal continuity.


The implementation of the RT space $R_{h, p}$ in NGSolve is accessed using `ng.HDiv`. The `Set` method, as in other finite element spaces in NGSolve, can be used to perform the interpolation. 

In [ ]:
p = 5   # This corresponds to the p in the RT definition above.
R = ng.HDiv(mesh, order=p, RT=True)

In [ ]:
qh = GridFunction(R)
qh.Set(q)
print('Error in RT interpolation =', 
      ng.sqrt(ng.Integrate((q - qh)**2, mesh)))

Compare this to what happens when we interpolate the same function in the product of two Lagrange finite element spaces.

In [ ]:
V2 = ng.VectorH1(mesh, order=p)  # Cartesian product of Lagrange spaces
qq = GridFunction(V2)
qq.Set(q)
print('Error in product Lagrange space interpolation =', 
      ng.sqrt(ng.Integrate((q - qq)**2, mesh)))

Clearly the error in the product space approach is several orders higher than that of the error produced by the RT space. The source of the large error while interpolating using  `ng.VectorH1` is immediately evident if you plot the $y$-component of the interpolant `qq` or the error.  The interpolant, being continuous in both components, has a difficult time approximating the discontinuous component.

In [ ]:
Draw(qq - q, mesh, 'Lagrange interpolation error', vectors={'grid_size': 25});

#### Shape functions

It is illustrative to visualize the basis functions of the RT space. We had gotten used to speaking of "hat functions" but that term is not useful for general finite elements. Basis functions generated from degrees of freedom of any abstract finite element spaces are called *shape functions*. Here is a visualization of a shape function of the *lowest order RT space* (the $p=0$ case).

In [ ]:
R = ng.HDiv(mesh, order=0, RT=True)

shapenumber = 15
shape = ng.GridFunction(R, name='shape')
shape.vec[:] = 0
shape.vec[shapenumber] = 1
Draw(shape, mesh, vectors={'grid_size': 20});

You can see any shape function you wish by revising the `shapenumber` in the code above. In all cases, you see that
- the shape functions are vector fields whose normal components are continuous, 
- they are supported on a patch of one or two elements, 
- their normal component is zero on all mesh facets except one.


Thus in the lowest order case, each shape function is associated to a single mesh facet (an edge,  in our 2D mesh). If this edge is on the domain boundary, the  shape function is supported on one triangle. If the edge is an interior edge, the  shape function is supported on the two triangles which share the interior edge.

One can also directly see the connection between the number of mesh facets and the dimension of the lowest order RT space by querying both numbers:

In [ ]:
mesh.nfacet

In [ ]:
R.ndof

Of course, for higher $p$, a number of additional shape functions must be added. You can easily visualize the higher order shape functions by tweaking the above few lines of code.

#### Convergence of divergence

Ordinarily, when a piecewise polynomial approximation $q_h$ to a function $q$ converges at a rate $O(h^k)$, we expect its derivatives to converge at a lower rate.

However,  somewhat miraculously, if the approximation $q_h$ is in the RT space, then $\dive(q_h - q)$ converges at the same rate as $q_h - q$.  

Please verify this yourself!

<div class="alert alert-info">
    <b><font color=red>Class Activity:</font></b>     
  Compute the $L^2$ norms of the interpolation errors and their divergences, for the case of the lowest order RT space $(p=0)$ and reproduce the above convergence rates.  (Start by computing the divergence of the previously given discontinuous vector field $q$.)
</div>

In a later lecture we will prove this superconvergence property of the divergence of the interpolation error when using the RT space. 

## The 2D Nedelec finite element space

Consider the curl operator in two space dimensions, acting on a vector field $u: \om \to \R^2$, 
$$
\newcommand{\curl}{\mathop{\mathrm{curl}}}
\curl u = \partial_0 u_1  - \partial_1 u_0.
$$
You have  worked with $\curl$ as an operator acting on three-dimensional (3D) vector fields. The above two-dimensional (2D) version is the obtained as the $z$-component of the 3D curl when applied to a vector field that has only $x$ and $y$ components with no $z$ dependence. It takes a 2D vector field $u$ and produces a scalar field $\curl u$.

The 2D curl is intimately related to the 2D divergence. To see this, let $J_{\pi/2}$ denote the operator that rotates a vector clockwise by 90 degrees, i.e., 
$$
J_{\pi/2} 
\begin{pmatrix}
a \\ b
\end{pmatrix} = 
\begin{pmatrix}
b \\ -a
\end{pmatrix}.
$$
Now, if two vector fields $u$ and $q$ are related by  
$$
u = J_{\pi/2} q 
$$
then obviously 
$$
\dive q =  \curl u.
$$

Moreover, if $t$ denotes the counterclockwise unit tangent vector on element boundaries, then it is related to the unit outward normal $n$ at the same boundary point by 
$$
J_{\pi/2} n = t.
$$
Hence 
$$
u\cdot t = q \cdot n.
$$
Thus the condition that   &LeftDoubleBracket; $q\cdot n$ &RightDoubleBracket; $= 0$
is equivalent to the condition that the *jump of the tangential component* of $u$ vanish, 
&LeftDoubleBracket; $u\cdot t$ &RightDoubleBracket; $= 0.$


We define the  Nedelec space in two dimensions as the RT space rotated:
$$
N_{h, p} := J_{\pi/2} R_{h,p}.  
$$
 It consists of piecewise polynomial vector fields that are tangentially continuous. Applying the rotation operator $J_{\pi/2}$ to the polynomial functions in our previous definition of $R_{h, p}$, we also find that $N_{h, p}$ can be equivalently described  by 
$$
N_{h, p} = 
\left\{
u\in H(\curl, \om) : \;\text{ for all elements }  K \in \oh, \quad 
u|_K \in P_p(K)^2 + \begin{pmatrix} -y \\ x \end{pmatrix} P_p(K) \; \right\}.
$$


By our prior discussion relating the 2D curl and div, we also conclude that $N_{h, p} \subset H(\curl, \om)$ since $R_{h,p} \subset H(\div, \om)$.    Note that this rotational isomorphism between $H(\dive, \om)$ and $H(\curl, \om)$ does not hold in 3D. The Nedelec space in 3D is a truly different space, not isomorphic to the 3D RT space. Postponing the 3D case for later discussions, let us proceed with the 2D case. NGSolve provides an implementation of the Nedelec space accessible through `HCurl` as follows.

In [ ]:
N = ng.HCurl(mesh, order=0, type1=True)

In [ ]:
shapenumber = 15
shape = GridFunction(N, name='shape')
shape.vec[:] = 0
shape.vec[shapenumber] = 1
Draw(shape, mesh, vectors={'grid_size': 30});

As you can see, this shape function visually appears to be exactly the same shape function plotted earlier for the RT case, except for a rotation by 90 degrees.  Note how the tangential component of the shape function varies continuously across the edges. Just as the RT space was useful for interpolating vector fields with normal continuity, the Nedelec space is useful for interpolating vector fields with tangential continuity. 

The Nedelec finite element is sometimes called the **edge finite element** in some engineering literature. Interestingly, the shape functions of the RT and the Nedelec element were known in other mathematical context (they appeared in a 1957 book by Whitney) long before these elements were known. In recognition of this, mathematicians sometimes refer to these shape functions as **Whitney forms**. Here is a formula for the shape functions (of the lowest order Nedelec space) that you just saw in the above visualization:

<div class="alert alert-info">
    <b><font color=red>Exercise I:</font></b>     
  Consider an edge $e_{ij}$ between adjacent mesh vertices $a_i , a_j$ of a triangular mesh. Let  $\lambda_i$ denote the barycentric coordinate function associated to the vertex $a_i$. Show that the function

$$\newcommand{\grad}{\mathop{\mathrm{grad}}}
    \phi_{ij} = \lambda_j \grad \lambda_i - \lambda_i \grad \lambda_j 
$$

has zero tangential components on all mesh edges except $e_{ij}$,  where it equals $ \phi_{ij} \cdot t_{ij} = 1/ |e_{ij}|$. Here  $|e_{ij}|$ is the length of $e_{ij}$ and  $t_{ij} = (a_i - a_j)/ |e_{ij}|$.
</div>

## Range of the 2D curl 

Application of the 2D curl operator to a function in the Nedelec space $N_{h, p}$ results in a function with no continuity restrictions across element interfaces in general. Since the application of curl also reduces the polynomial degree within each element, the result is a piecewise polynomial of degree not more than $p$, i.e., a function in the DG space $P_p(\oh)$. Thus, we may view $\curl$ as an operator from the Nedelec space to the DG space, 
$$
\curl : N_{h, p}  \to P_{p}(\oh).
$$
*How much of the DG space is filled with the range of $\curl$?*

To answer this,  we will need to understand the range of the above map. Since $\curl$ may act on smooth function spaces or on finite element spaces like  $N_{h, p}$, it is a good idea to disambiguate the specific domain we consider.  So we will write 
$
\text{`` }\curl: N_{h,p}  \text{ ''}
$
to indicate the $\curl$ when it's viewed as an operator acting (only) on $N_{h, p}$ functions. We now proceed to compute the dimension of its range, i.e., the number 
$$\newcommand{\rank}{\mathop{\mathrm{rank}}}
\rank(\curl: N_{h, p}).
$$
To do so, we will need a representation of the operator $\curl: N_{h,p}$  as a matrix obtained using the basis functions of $N_{h,p}$ and $P_{p-1}(\oh)$. Of course, we will also need a domain and a mesh.


TO BE CONTINUED



<hr>
    
    
$\ll$ [Table Of Contents](./INDEX.ipynb)
<br>
$\ll$ [Jay Gopalakrishnan](http://web.pdx.edu/~gjay/)

    

